In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sklearn

In [2]:
data=pd.read_csv('Data.csv')

In [3]:
data.head()

,RANK,CANDIDATE NAME,GENDER,REGION,CATEGORY,SEAT CATEGORY,BRANCH,COLLEGE
0,144,KOTA VIGNAN,M,OU,BC_B,OC_GEN_UR,CSE,MVSR
1,233,SONABOYINA NIKHIL TEJA,M,OU,BC_D,OC_GEN_UR,CSE,MVSR
2,243,RACHAKONDA ESHWAR,M,OU,BC_B,BC_B_GEN_UR,CSE,MVSR
3,286,CHALLA RUPAK VARDHAN,M,OU,OC,EWS_GEN_UR,CSE,MVSR
4,318,SHRIYANS MEDIPELLY,M,OU,BC_D,BC_D_GEN_OU,CSE,JNTH


In [4]:
data['RANK'].value_counts()

50470     3
115496    2
92409     2
16896     2
39205     2
         ..
33854     1
33856     1
33861     1
33862     1
300293    1
Name: RANK, Length: 49310, dtype: int64

In [5]:
data.drop_duplicates(['RANK'],inplace=True)

In [6]:
data[data['RANK'] == 10101]

,RANK,CANDIDATE NAME,GENDER,REGION,CATEGORY,SEAT CATEGORY,BRANCH,COLLEGE
4237,10101,AYYAGARI ANNAPURNA NAGA MAHESREE,F,OU,OC,OC_NCC_GIRLS_OU(57),ECE,VASV


In [7]:
data['CATEGORY'].value_counts()

OC      11148
BC_B    10832
BC_D     8441
SC       7538
ST       4503
BC_A     3536
BC_E     3032
BC_C      280
Name: CATEGORY, dtype: int64

In [8]:
data['COLLEGE'].value_counts()

VJEC    1286
SNIS    1028
IARE    1025
CBIT     936
MREC     936
        ... 
GLOB      66
TCEK      60
MNRT      51
BSKR      41
PLMU      18
Name: COLLEGE, Length: 126, dtype: int64

In [9]:
data['REGION'].value_counts()

OU     47426
AU      1148
SVU      565
NL       171
Name: REGION, dtype: int64

In [10]:
# Combine 'BRANCH' and 'COLLEGE' columns into 'target'
data['target'] = data['COLLEGE'] + '_' + data['BRANCH']

# Drop 'CANDIDATE NAME' column
data.drop(['CANDIDATE NAME','COLLEGE','BRANCH','GENDER', 'REGION', 'CATEGORY',], axis=1, inplace=True)

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
le=LabelEncoder()
data['target']=le.fit_transform(data['target'])
#columns_to_encode = ['GENDER', 'REGION', 'CATEGORY', 'SEAT CATEGORY']
columns_to_encode = ['SEAT CATEGORY']

print(data.head())
# Define the columns to be encoded and the columns to be skipped
transfor = Pipeline([
    ('encoding', ColumnTransformer([('one_hot_encoding', OneHotEncoder(), columns_to_encode)], remainder='passthrough'))
])

data_trans=transfor.fit_transform(data.drop('target',axis=1))

   RANK SEAT CATEGORY  target
0   144     OC_GEN_UR     534
1   233     OC_GEN_UR     534
2   243   BC_B_GEN_UR     534
3   286    EWS_GEN_UR     534
4   318   BC_D_GEN_OU     291


In [12]:
data_trans

<49310x197 sparse matrix of type '<class 'numpy.float64'>'
	with 98620 stored elements in Compressed Sparse Row format>

In [17]:
model=Pipeline([("clf",KNeighborsClassifier())])

from skopt import BayesSearchCV
from skopt.space import Real,Integer

search_space = {
    'clf__algorithm': ['auto', 'brute'],
    'clf__leaf_size': Integer(1,100),
    'clf__n_neighbors': Integer(1,80),
    'clf__metric': ['cityblock', 'euclidean', 'l1', 'l2', 'manhattan','cosine'],
    'clf__p': Integer(0, 3),
    'clf__weights': ['uniform','distance'],}

opt = BayesSearchCV(model, search_space, cv=2, n_iter=15, scoring='accuracy') 

opt.fit(data_trans,data['target'])

c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavi

BayesSearchCV(cv=2, estimator=Pipeline(steps=[('clf', KNeighborsClassifier())]),
              n_iter=15, scoring='accuracy',
              search_spaces={'clf__algorithm': ['auto', 'brute'],
                             'clf__leaf_size': Integer(low=1, high=100, prior='uniform', transform='normalize'),
                             'clf__metric': ['cityblock', 'euclidean', 'l1',
                                             'l2', 'manhattan', 'cosine'],
                             'clf__n_neighbors': Integer(low=1, high=80, prior='uniform', transform='normalize'),
                             'clf__p': Integer(low=0, high=3, prior='uniform', transform='normalize'),
                             'clf__weights': ['uniform', 'distance']})

In [19]:
best=opt.best_estimator_
best.fit(data_trans,data['target'])

Pipeline(steps=[('clf',
                 KNeighborsClassifier(algorithm='brute', leaf_size=100,
                                      metric='cosine', n_neighbors=76, p=0))])

Pipeline(steps=[('clf',
                 KNeighborsClassifier(algorithm='brute', leaf_size=77,
                                      metric='cosine', n_neighbors=10, p=1,
                                      weights='distance'))])

In [20]:
test=[[20000,'OC_GIRLS_OU']]  #20k rank, girl
test=pd.DataFrame(test,columns=['RANK','SEAT CATEGORY'])
test_trans=transfor.transform(test)
print(test_trans[0])
le.inverse_transform( best.predict(test_trans) )

  (0, 118)	1.0
  (0, 196)	20000.0


c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array(['STLW_CSE'], dtype=object)

In [21]:
lofpro=list(best.predict_proba(test_trans))
for i in range(5):
    max=np.argmax(lofpro)
    print(f'{le.inverse_transform([max])} {lofpro[0][max]*100} % probable admission')
    lofpro[0]=np.delete(lofpro[0],max)

['STLW_CSE'] 9.210526315789473 % probable admission
['TKRC_CSD'] 6.578947368421052 % probable admission
['MLID_CSM'] 5.263157894736842 % probable admission
['MRCW_CSE'] 5.263157894736842 % probable admission
['BREW_INF'] 3.9473684210526314 % probable admission
